In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import random
from PIL import Image
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator
from sklearn import metrics as met
from sklearn import model_selection as ms


Using TensorFlow backend.


In [2]:
! rm -R CNN_SLIKE
! rm -R Hey-Waldo
! rm -R SLIKE_AUG
! rm -R ml03-finding-wally

! git clone https://github.com/vc1492a/Hey-Waldo.git
! git clone https://github.com/matf-ml/ml03-finding-wally.git

! mkdir CNN_SLIKE
! mkdir CNN_SLIKE/waldo
! mkdir CNN_SLIKE/Waldo_transparent_64_64
! mv Hey-Waldo/64/notwaldo CNN_SLIKE
! mv ml03-finding-wally/data/Slike/Waldo_transparent CNN_SLIKE

Cloning into 'Hey-Waldo'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 18808 (delta 0), reused 1 (delta 0), pack-reused 18807
Receiving objects: 100% (18808/18808), 112.65 MiB | 36.83 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Checking out files: 100% (20839/20839), done.
Cloning into 'ml03-finding-wally'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 369 (delta 0), reused 19 (delta 0), pack-reused 347
Receiving objects: 100% (369/369), 166.13 MiB | 40.10 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [0]:
# Smanjivanje slika transparentnog Walda na 64x64

gcolab_putanja_notwaldo = os.path.join ('CNN_SLIKE', 'notwaldo')
gcolab_putanja_waldo = os.path.join ('CNN_SLIKE', 'waldo')
gcolab_putanja_transp = os.path.join ('CNN_SLIKE', 'Waldo_transparent')
gcolab_putanja_transp_64 = os.path.join ('CNN_SLIKE', 'Waldo_transparent_64_64')

slike_transp = [os.path.join(gcolab_putanja_transp, f) for f in os.listdir(gcolab_putanja_transp) if os.path.isfile(os.path.join(gcolab_putanja_transp, f))]

# Slika sa Waldom smanjujemo na 64x64

size = (64, 64, 4)
i = 1

for slika in slike_transp:
    
    ime = '%d.png' % i
    izlazna_putanja = os.path.join ('CNN_SLIKE', 'Waldo_transparent_64_64', ime)
    i += 1
    
    if slika != izlazna_putanja:
        try:
            im = Image.open (slika)
            im.thumbnail(size, Image.ANTIALIAS)
            im.save(izlazna_putanja, "PNG")
            
        except IOError:
            print ("Greska pri kreiranju '%s'" % slika)


In [0]:
# Lepljenje transparentnih slika sa likom Walda na slike iz notwaldo skupa

# Iteriramo kroz notwaldo slike i lepimo na njih transparentnog walda

slike_transp_64 = [os.path.join(gcolab_putanja_transp_64, f) for f in os.listdir(gcolab_putanja_transp_64) if os.path.isfile(os.path.join(gcolab_putanja_transp_64, f))]
slike_notwaldo = [os.path.join(gcolab_putanja_notwaldo, f) for f in os.listdir(gcolab_putanja_notwaldo) if os.path.isfile(os.path.join(gcolab_putanja_notwaldo, f))]

i = 0
num_generated_files = 0
for slika_1 in slike_notwaldo:
    
    pozadina = Image.open(slika_1).convert('RGBA')
    
    # Kroz Waldo slike idemo ciklicno i lepimo redom
    valdo_slika = Image.open(slike_transp_64 [i % len(slike_transp_64)]).convert('RGBA')
    
    poz_1 = random.randrange (0, pozadina.size[0] - valdo_slika.size[0] + 1, 1)
    poz_2 = random.randrange (0, pozadina.size[1] - valdo_slika.size[1] + 1, 1)
    
    pozadina.paste(valdo_slika, (poz_1, poz_2), valdo_slika)
    
    new_file_path = '%s/aug_%s.jpeg' % (gcolab_putanja_waldo, num_generated_files)
    pozadina.convert('RGB').save(new_file_path, "JPEG")
    
    num_generated_files += 1
    i += 1


In [0]:
! mkdir SLIKE_AUG
! mkdir SLIKE_AUG/waldo
! mkdir SLIKE_AUG/notwaldo

In [6]:
# Koriscenje ugradjene funkcije za generisanje novih podataka

gcolab_waldo_ulaz = os.path.join ('CNN_SLIKE')
gcolab_notwaldo_ulaz = os.path.join ('CNN_SLIKE')
gcolab_waldo_izlaz = os.path.join ('SLIKE_AUG', 'waldo')
gcolab_notwaldo_izlaz = os.path.join ('SLIKE_AUG', 'notwaldo')


osobine = ImageDataGenerator (
        rotation_range = 180,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        zoom_range=0.2,
        fill_mode = 'nearest')


# Generisanje Waldo slika
generator_1 = osobine.flow_from_directory(
    directory = gcolab_waldo_ulaz,
    save_to_dir = gcolab_waldo_izlaz,
    save_prefix = 'gen',
    save_format = 'jpeg',
    target_size = (64, 64),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = 'binary',
    classes = ['waldo'],
    seed=3
)

# Generisanje notwaldo slika
generator_2 = osobine.flow_from_directory(
    directory = gcolab_notwaldo_ulaz,
    save_to_dir = gcolab_notwaldo_izlaz,
    save_prefix = 'gen',
    save_format = 'jpeg',
    target_size = (64, 64),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = 'binary',
    classes = ['notwaldo'],
    seed=3
)

granica = 5000

# Pokrenuti ako je potrebno generisati nove slike

for i in range (granica):
    generator_1.next()
    generator_2.next()



Found 5337 images belonging to 1 classes.
Found 5337 images belonging to 1 classes.


In [7]:
! ls -l SLIKE_AUG/waldo | wc -l
! ls -l SLIKE_AUG/notwaldo | wc -l

159798
159798


In [0]:
# Kreiranje numpy nizova od dobijenih slika

gcolab_valdo_putanja = gcolab_waldo_izlaz
gcolab_ne_valdo_putanja = gcolab_notwaldo_izlaz

valdo = np.array([np.array(Image.open(os.path.join(gcolab_putanja_waldo, ime_slike))) for ime_slike in os.listdir(gcolab_putanja_waldo)])
ne_valdo = np.array([np.array(Image.open(os.path.join(gcolab_putanja_notwaldo, ime_slike))) for ime_slike in os.listdir(gcolab_putanja_notwaldo)])


In [0]:
# Deljenje na skupove

y1 = [1 for i in range(len(valdo))]
y2 = [0 for i in range(len(ne_valdo))]
y = np.array(y1 + y2)

X = np.concatenate ((valdo, ne_valdo))

x_trening_valid, x_test, y_trening_valid, y_test = ms.train_test_split(X, y, train_size = 0.67, test_size=0.33, stratify = y, random_state = 3)
x_trening, x_valid, y_trening, y_valid = ms.train_test_split(x_trening_valid, y_trening_valid, train_size = 0.7, test_size = 0.3, stratify = y_trening_valid, random_state = 3)


In [0]:
# Centriranje

x_trening = x_trening / 255
x_trening_valid = x_trening_valid / 255
x_test = x_test / 255


In [11]:
# Kreiranje modela

knn = Sequential()

knn.add(Conv2D(filters = 32, 
               kernel_size = (2, 2), 
               strides = (1,1),
               padding = 'same',
               input_shape = (64, 64, 3),
               data_format = 'channels_last'))

knn.add(Activation('relu'))

knn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

knn.add(Dropout(0.4))

knn.add(Conv2D(filters = 64,
               kernel_size = (2,2),
               strides = (1,1),
               padding = 'valid'))

knn.add(Activation('relu'))

knn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

knn.add(Flatten())

knn.add(Dense(64))

knn.add(Activation('relu'))

knn.add(Dropout(0.4))

knn.add(Dense(1))

knn.add(Activation('sigmoid'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# Treniranje modela

knn.compile (optimizer = Adam(lr=0.00075), loss='binary_crossentropy', metrics=['accuracy'])

knn.fit (x = x_trening, y = y_trening, batch_size=64, epochs=200, validation_data = (x_valid, y_valid))



Instructions for updating:
Use tf.cast instead.
Train on 5005 samples, validate on 2146 samples
Epoch 1/200
5005/5005 [==============================] - 4s 840us/step - loss: 0.6953 - acc: 0.5592 - val_loss: 4.8107 - val_acc: 0.6678
Epoch 2/200
5005/5005 [==============================] - 1s 222us/step - loss: 0.5277 - acc: 0.7299 - val_loss: 3.3909 - val_acc: 0.7791
Epoch 3/200
5005/5005 [==============================] - 1s 221us/step - loss: 0.3758 - acc: 0.8154 - val_loss: 3.1558 - val_acc: 0.7861
Epoch 4/200
5005/5005 [==============================] - 1s 226us/step - loss: 0.3057 - acc: 0.8428 - val_loss: 5.1086 - val_acc: 0.6719
Epoch 5/200
5005/5005 [==============================] - 1s 226us/step - loss: 0.2683 - acc: 0.8597 - val_loss: 4.6644 - val_acc: 0.6966
Epoch 6/200
5005/5005 [==============================] - 1s 224us/step - loss: 0.2331 - acc: 0.8721 - val_loss: 5.8688 - val_acc: 0.6300
Epoch 7/200
5005/5005 [==============================] - 1s 221us/step - loss: 0.2

In [13]:
# Evaluacija modela

greska, tacnost = knn.evaluate(x_test, y_test)

print(f'Greska: {greska}, tacnost: {tacnost}')


3523/3523 [==============================] - 0s 122us/step
Greska: 0.00264895189130515, tacnost: 0.999716151007664


In [14]:
# Treniranje modela na celom skupu podataka

knn.compile (optimizer = Adam(lr=0.00075), loss='binary_crossentropy', metrics=['accuracy'])

knn.fit (x = X, y = y, batch_size=64, epochs=400)



Epoch 1/400
10674/10674 [==============================] - 2s 205us/step - loss: 2.3064 - acc: 0.8360
Epoch 2/400
10674/10674 [==============================] - 2s 167us/step - loss: 2.8389 - acc: 0.8005
Epoch 3/400
10674/10674 [==============================] - 2s 168us/step - loss: 2.9710 - acc: 0.8042
Epoch 4/400
10674/10674 [==============================] - 2s 168us/step - loss: 1.8169 - acc: 0.8811
Epoch 5/400
10674/10674 [==============================] - 2s 167us/step - loss: 2.4419 - acc: 0.8413
Epoch 6/400
10674/10674 [==============================] - 2s 167us/step - loss: 2.6764 - acc: 0.8227
Epoch 7/400
10674/10674 [==============================] - 2s 168us/step - loss: 1.7096 - acc: 0.8849
Epoch 8/400
10674/10674 [==============================] - 2s 167us/step - loss: 0.9902 - acc: 0.9240
Epoch 9/400
10674/10674 [==============================] - 2s 168us/step - loss: 1.0455 - acc: 0.9234
Epoch 10/400
10674/10674 [==============================] - 2s 168us/step - loss: 

In [0]:
# Testiranje za neku originalnu sliku

def izdeli_sliku (put_slike, put_izlaza):
    
    slika = Image.open(put_slike)
    ime_slike = str.split (put_slike, sep = os.path.sep)[-1]
    sir_slike, vis_slike = slika.size
    
    print (ime_slike)
    
    for i in range (0, sir_slike, 64):
        
        for j in range (0, vis_slike, 64):
            
            if i + 64 < sir_slike:
                k = i + 64
            else:
                i = sir_slike - 64 - 1
                k = sir_slike - 1
            
            if j + 64 < vis_slike:
                l = j + 64
            else:
                j = vis_slike - 64 - 1
                l = vis_slike - 1
            
            
            kontura = (i, j, k, l)
            isecak = slika.crop (kontura)
            
            
            try:
              isecak.save (os.path.join(put_izlaza, str.split (ime_slike, sep = '.')[-2] + "_%s_" % str(i) + "%s.jpg" % str(j)))
                
            except:
                print ('Nije uspelo cuvanje slike!')


In [0]:

! rm -R TEST
! rm -R REZULTATI
! mkdir TEST
! mkdir REZULTATI

In [17]:
put_slike = os.path.join ('Hey-Waldo', 'original-images', '3.jpg')
put_izlaza = os.path.join ('TEST')

izdeli_sliku (put_slike, put_izlaza)

3.jpg


In [0]:
test = np.array([np.array(Image.open(os.path.join(put_izlaza, ime_slike))) for ime_slike in os.listdir(put_izlaza)])


In [19]:
niz_rezultata = knn.predict (test, batch_size = 64, verbose = 1)


704/704 [==============================] - 0s 180us/step


mkdir: cannot create directory ‘REZULTATI’: File exists


In [21]:
put_rezultata = os.path.join ('REZULTATI')

kandidati = []

for i in range (len(niz_rezultata)):
  if niz_rezultata[i] != 0:
    print (i, niz_rezultata[i])
    kandidati.append (Image.fromarray (test[i]))


for i in range (len(kandidati)):
  ime = '%d.jpg' % i
  kandidati[i].save(os.path.join (put_rezultata, ime))


20 [0.63837457]
64 [1.]
125 [1.]
155 [0.63837457]
205 [1.]
214 [1.]
217 [0.63837457]
286 [0.63837457]
323 [0.63837457]
332 [0.63837457]
385 [1.]
411 [1.]
484 [1.]
507 [0.63837457]
539 [1.]
553 [1.]
601 [1.]
665 [1.]
